In [1]:
# Download github repository
!git clone https://github.com/uw-cmg/RPV_model_benchmarking

Cloning into 'RPV_model_benchmarking'...
remote: Enumerating objects: 446, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 446 (delta 9), reused 5 (delta 2), pack-reused 428 (from 3)
Receiving objects: 100% (446/446), 783.81 MiB | 35.28 MiB/s, done.
Resolving deltas: 100% (78/78), done.
Updating files: 100% (447/447), done.


In [2]:
!pip install -r RPV_model_benchmarking/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
# Restart the session (Runtime -> Restart Session) so the new installs can take effect

In [1]:
# Package imports
import pandas as pd
from RPV_model_benchmarking.models import *
from RPV_model_benchmarking.data import *
from RPV_model_benchmarking.metrics import *
from RPV_model_benchmarking.plots import *

scikit-lego is an optional dependency, enabling use of the LowessRegression model. If you want to use this model, do "pip install scikit-lego"
linear-tree is an optional dependency, enabling use of Linear tree, forest, and boosting models. If you want to use this model, do "pip install linear-tree"
gplearn is an optional dependency, enabling the use of genetic programming SymbolicRegressor model. If you want to use this model, do "pip install gplearn"


In [ ]:
# Make all the model predictions on the full dataset

df = DataLoader().load_rpv_data()

preds, df = EONY().predict(df)

preds, df = OWAY().predict(df)

preds, df = JOWAY().predict(df, atr2fte=1.38e20, nn_model='Jacobs25')

preds, df = E900().predict(df)

preds, df = GBR().predict(df)

preds, df = GKRR().predict(df)

preds, df = EnsembleNN_Jacobs23().predict(df, return_ebars=False)

preds, df = EnsembleNN_Jacobs24().predict(df, return_ebars=False)

preds, df = EnsembleNN_Jacobs25().predict(df, return_ebars=False)


Could not find designated model type in scikit-learn model library. Note the other supported modeltype is the keras.wrappers.scikit_learn.KerasRegressor model
  1/142 ━━━━━━━━━━━━━━━━━━━━ 12s 91ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(


142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


/content/RPV_model_benchmarking/models.py:462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['Product Form_0'] = pf_0
/content/RPV_model_benchmarking/models.py:463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_features['Product Form_1'] = pf_1
/content/RPV_model_benchmarking/models.py:464: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

Could not find designated model type in scikit-learn model library. Note the other supported modeltype is the keras.wrappers.scikit_learn.KerasRegressor model
 13/142 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 

...

142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Could not find designated model type in scikit-learn model library. Note the other supported modeltype is the keras.wrappers.scikit_learn.KerasRegressor model
 15/142 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(


142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Could not find designated model type in scikit-learn model library. Note the other supported modeltype is the keras.wrappers.scikit_learn.KerasRegressor model
  1/142 ━━━━━━━━━━━━━━━━━━━━ 11s 83ms/step

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(


142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [ ]:
# Just keep high fluence data and save the preds file

df = Metrics().filter_df(df, column_name='fluence_n_cm2', filter_val=6e19, filter_operation='greater')
df = Metrics().filter_df(df, column_name='datatype', filter_val='Plotter', filter_operation='equal')
df

,temperature_C,wt_percent_Cu,wt_percent_Ni,wt_percent_Mn,wt_percent_P,wt_percent_Si,wt_percent_C,log(fluence_n_cm2),log(flux_n_cm2_sec),Product Form,...,JOWAY predicted TTS (degC),E900 predicted TTS (degC),GBR predicted TTS (degC),GKRR predicted TTS (degC),Jacobs23 NN ensemble predicted TTS (degC),Jacobs23 NN ensemble error bars (degC),Jacobs24 NN ensemble predicted TTS (degC),Jacobs24 NN ensemble error bars (degC),Jacobs25 NN ensemble predicted TTS (degC),Jacobs25 NN ensemble error bars (degC)
55,298.888889,0.100,0.180,1.42,0.016,0.25,0.300,20.029384,12.210868,P,...,66.510078,68.744293,89.739510,83.623600,75.749794,NaN,73.508690,NaN,73.552902,NaN
181,262.000000,0.087,0.640,1.26,0.017,0.29,0.176,19.828660,11.240388,F,...,97.433243,100.948511,104.024582,104.900951,100.453285,NaN,106.863831,NaN,108.017807,NaN
182,262.000000,0.087,0.640,1.26,0.017,0.29,0.176,20.053078,11.464807,F,...,126.377009,129.712560,147.618860,144.893409,131.221390,NaN,145.510284,NaN,144.109833,NaN
183,262.918657,0.090,0.650,1.27,0.010,0.31,0.169,20.035029,11.361087,F,...,122.095163,116.748711,147.205189,145.612038,133.781479,NaN,143.025055,NaN,139.684845,NaN
184,262.918657,0.090,0.650,1.27,0.010,0.31,0.169,20.064832,11.390890,F,...,126.174598,120.642986,146.256762,147.869161,137.684875,NaN,147.351410,NaN,144.002411,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,292.777778,0.070,0.590,1.45,0.007,0.24,0.200,19.780317,10.858921,P,...,47.175323,47.136852,43.965144,48.674868,43.442955,NaN,46.127247,NaN,43.742363,NaN
1822,292.777778,0.070,0.590,1.45,0.007,0.24,0.200,19.780317,10.858921,P,...,47.175323,47.136852,43.965144,48.674868,43.442955,NaN,46.127239,NaN,43.742363,NaN
1823,292.777778,0.047,0.049,1.28,0.009,0.14,0.150,19.780317,10.858921,W,...,31.053662,35.596852,26.838923,19.497101,24.867025,NaN,23.197323,NaN,26.158789,NaN
1895,287.000000,0.064,0.690,1.32,0.006,0.22,0.138,19.841985,11.299159,F,...,58.307228,48.540352,48.563905,51.950073,56.244244,NaN,54.366199,NaN,56.828743,NaN


In [ ]:
df.to_csv('RPV_preds_highfluence.csv', index=False)

In [ ]:
# Get model residuals and noise plot

print(df.shape)
df = Metrics().filter_df(df, column_name='Measured DT41J  [C]', filter_val=50, filter_operation='greater')
print(df.shape)

plot_residuals(trues=df['Measured DT41J  [C]'],
               preds=df['Jacobs25 NN ensemble predicted TTS (degC)'],
               name='Jacobs25')

plot_residuals(trues=df['Measured DT41J  [C]'],
               preds=df['Jacobs23 NN ensemble predicted TTS (degC)'],
               name='Jacobs23')

plot_model_noise(trues=df['Measured DT41J  [C]'], noise=9, num_points=df.shape[0])

In [ ]:
# Get the 5fold CV benchmarking results

data = Benchmarking().get_5fold_benchmarks(model_name='Jacobs25')
data

In [ ]:
# Make best and worst alloy ranking plot

df_best, df_best_sort = plot_best_worst_alloys(model_name='Jacobs25', num_alloys=25, metric='RMSE', plot_type='best')

df_worst, df_worst_sort = plot_best_worst_alloys(model_name='Jacobs25', num_alloys=25, metric='RMSE', plot_type='worst')

In [ ]:
# Make embrittlement curve

plot_embrittlement_curve(df,
                         model_name_list=['Jacobs23', 'Jacobs24', 'Jacobs25'],
                         flux_list=[3*10**10, 3.68*10**12],
                         style='linear',
                         num_points=50,
                         ymax=350,
                        include_ATR2=True)

In [ ]:
# Make flux effect crossover histogram

df = DataLoader().load_rpv_data()

model_name_list = ['Jacobs23', 'Jacobs24', 'Jacobs25']
fluence = 6*10**19 # 6*10**19, 1*10**20
plot_flux_effect_crossover_histogram(df, model_name_list, fluence=fluence, flux_plotter=3*10**10, flux_atr2=3.68*10**12)

In [ ]:
# Make crossplots

df = DataLoader().load_rpv_data()

plot_crossplot(df,
               #model_name_list=['GKRR'],
               #crossplot_variable_name = 'at_percent_Cu',
               crossplot_variable_name='wt_percent_Cu',
               model_name_list=['GBR', 'OWAY', 'Jacobs23'],
               crossplot_variable_min=0,
               crossplot_variable_max=0.8,
               grid_variables = ['wt_percent_Ni'],
               grid_values = [[0.4, 0.8, 1.2]],
               #grid_variables=['log(flux_n_cm2_sec)', 'wt_percent_Ni'],
               #grid_values=[[np.log10(3e10), np.log10(3.68e12)], [0.4, 0.8]],
               num_points=50,
               ymax=250)